# Document Transformers

In [34]:
from langchain_text_splitters import CharacterTextSplitter

text = """The world of street food is a vibrant tapestry of flavors and cultures. From sizzling skewers to spicy tacos, every corner of the globe has its own unique offerings. Vendors often  up shop in bustling markets or on busy street corners, attracting hungry passersby with mouthwatering aromas.

One of the best things about street food is its accessibility. It’s quick, affordable, and often made fresh right in front of you. Whether it’s a steaming bowl of pho in Vietnam or a crispy samosa in India, there’s something for everyone to enjoy.

Street food also brings people together. Friends and families gather around food stalls, sharing dishes and stories. It’s a social experience that transcends language and culture, creating connections over a shared love of good eats.

Finally, street food is constantly evolving. Chefs are experimenting with traditional recipes, adding modern twists and fusion flavors. This creativity keeps the scene exciting and ensures there’s always something new to try."""



## Length Based Splitter

- Token-based: Splits text based on the number of tokens, which is useful when working with language models.
- Character-based: Splits text based on the number of characters, which can be more consistent across different types of text.




In [35]:

# text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=300, chunk_overlap=10)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=100, chunk_overlap=20
)


docs = text_splitter.create_documents([text])

print(type(docs))

print(len(docs))

print(docs[0])


<class 'list'>
3
page_content='The world of street food is a vibrant tapestry of flavors and cultures. From sizzling skewers to spicy tacos, every corner of the globe has its own unique offerings. Vendors often  up shop in bustling markets or on busy street corners, attracting hungry passersby with mouthwatering aromas.'


## Text-strucutre based splitter

- The RecursiveCharacterTextSplitter attempts to keep larger units (e.g., paragraphs) intact.
- If a unit exceeds the chunk size, it moves to the next level (e.g., sentences).
- This process continues down to the word level if necessary.

In [36]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)

docs = text_splitter.create_documents([text])

print(type(docs))

print(len(docs))

print(docs[0])

<class 'list'>
8
page_content='The world of street food is a vibrant tapestry of flavors and cultures. From sizzling skewers to spicy tacos, every corner of the globe has its own unique offerings. Vendors often  up shop in bustling'


## Document-structured based
- Preserves the logical organization of the document
- Maintains context within each chunk
- Can be more effective for downstream tasks like retrieval or summarization

**Examples of structure-based splitting:**
- Markdown: Split based on headers (e.g., #, ##, ###)
- HTML: Split using tags
- JSON: Split by object or array elements
- Code: Split by functions, classes, or logical blocks

### Markdown Splitter


In [37]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

# read markdown file
with open("data/sample.md", "r") as f:
    md_text = f.read()
    
md_text

'# Title of the document\n\nThis is a sample document. It is written in markdown format.\n\n## Section 1\n\nThis is the first section of the document.\n\n### Subsection 1.1\n\nThis is the first subsection of the document.\n\n### Subsection 1.2\n\nThis is the second subsection of the document.\n\n## Section 2\n\nThis is the second section of the document.\n\n\n## Section 3\n\nThis is the third section of the document.\n\n\n\n'

In [38]:

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)

docs = markdown_splitter.split_text(md_text)

print(type(docs))

print(len(docs))

docs



<class 'list'>
6


[Document(metadata={'Header 1': 'Title of the document'}, page_content='# Title of the document  \nThis is a sample document. It is written in markdown format.'),
 Document(metadata={'Header 1': 'Title of the document', 'Header 2': 'Section 1'}, page_content='## Section 1  \nThis is the first section of the document.'),
 Document(metadata={'Header 1': 'Title of the document', 'Header 2': 'Section 1', 'Header 3': 'Subsection 1.1'}, page_content='### Subsection 1.1  \nThis is the first subsection of the document.'),
 Document(metadata={'Header 1': 'Title of the document', 'Header 2': 'Section 1', 'Header 3': 'Subsection 1.2'}, page_content='### Subsection 1.2  \nThis is the second subsection of the document.'),
 Document(metadata={'Header 1': 'Title of the document', 'Header 2': 'Section 2'}, page_content='## Section 2  \nThis is the second section of the document.'),
 Document(metadata={'Header 1': 'Title of the document', 'Header 2': 'Section 3'}, page_content='## Section 3  \nThis is 

In [39]:
# Char-level splits
from attr import s
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size = 10
chunk_overlap = 2
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(docs)
splits

[Document(metadata={'Header 1': 'Title of the document'}, page_content='# Title of'),
 Document(metadata={'Header 1': 'Title of the document'}, page_content='the'),
 Document(metadata={'Header 1': 'Title of the document'}, page_content='document'),
 Document(metadata={'Header 1': 'Title of the document'}, page_content='This is a'),
 Document(metadata={'Header 1': 'Title of the document'}, page_content='a sample'),
 Document(metadata={'Header 1': 'Title of the document'}, page_content='document.'),
 Document(metadata={'Header 1': 'Title of the document'}, page_content='It is'),
 Document(metadata={'Header 1': 'Title of the document'}, page_content='written'),
 Document(metadata={'Header 1': 'Title of the document'}, page_content='in'),
 Document(metadata={'Header 1': 'Title of the document'}, page_content='markdown'),
 Document(metadata={'Header 1': 'Title of the document'}, page_content='format.'),
 Document(metadata={'Header 1': 'Title of the document', 'Header 2': 'Section 1'}, page_

### HTML Splitter
#### Header Splitter

In [40]:
from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
    <!DOCTYPE html>
    <html>
    <body>
        <div>
            <h1>Foo</h1>
            <p>Some intro text about Foo.</p>
            <div>
                <h2>Bar main section</h2>
                <p>Some intro text about Bar.</p>
                <h3>Bar subsection 1</h3>
                <p>Some text about the first subtopic of Bar.</p>
                <h3>Bar subsection 2</h3>
                <p>Some text about the second subtopic of Bar.</p>
            </div>
            <div>
                <h2>Baz</h2>
                <p>Some text about Baz</p>
            </div>
            <br>
            <p>Some concluding text about Foo</p>
        </div>
    </body>
    </html>
"""


headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

[Document(metadata={}, page_content='Foo'),
 Document(metadata={'Header 1': 'Foo'}, page_content='Some intro text about Foo.  \nBar main section Bar subsection 1 Bar subsection 2'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section'}, page_content='Some intro text about Bar.'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 1'}, page_content='Some text about the first subtopic of Bar.'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 2'}, page_content='Some text about the second subtopic of Bar.'),
 Document(metadata={'Header 1': 'Foo'}, page_content='Baz'),
 Document(metadata={'Header 1': 'Foo', 'Header 2': 'Baz'}, page_content='Some text about Baz'),
 Document(metadata={'Header 1': 'Foo'}, page_content='Some concluding text about Foo')]

#### Section Splitter

In [41]:
from langchain_text_splitters import HTMLSectionSplitter



headers_to_split_on = [("h1", "Header 1"), ("h2", "Header 2")]

html_splitter = HTMLSectionSplitter(headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

[Document(metadata={'Header 1': 'Foo'}, page_content='Foo \n Some intro text about Foo.'),
 Document(metadata={'Header 2': 'Bar main section'}, page_content='Bar main section \n Some intro text about Bar. \n Bar subsection 1 \n Some text about the first subtopic of Bar. \n Bar subsection 2 \n Some text about the second subtopic of Bar.'),
 Document(metadata={'Header 2': 'Baz'}, page_content='Baz \n Some text about Baz \n \n \n Some concluding text about Foo')]

## Semantic Meaning Based

- Start with the first few sentences and generate an embedding.
- Move to the next group of sentences and generate another embedding (e.g., using a sliding window approach).
- Compare the embeddings to find significant differences, which indicate potential "break points" between semantic sections.


In [42]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import OllamaEmbeddings

embedding_model = OllamaEmbeddings(model="mxbai-embed-large")
# embeddings: Embeddings,
# buffer_size: int = 1,
# add_start_index: bool = False,
# breakpoint_threshold_type: BreakpointThresholdType = "percentile",
# breakpoint_threshold_amount: Optional[float] = None,
# number_of_chunks: Optional[int] = None,
# sentence_split_regex: str = r"(?<=[.?!])\s+",
# min_chunk_size: Optional[int] = None,
text_splitter = SemanticChunker(embedding_model,number_of_chunks=3)

docs = text_splitter.create_documents([text])

print(type(docs))

print(len(docs))


docs



<class 'list'>
3


[Document(metadata={}, page_content='The world of street food is a vibrant tapestry of flavors and cultures. From sizzling skewers to spicy tacos, every corner of the globe has its own unique offerings. Vendors often  up shop in bustling markets or on busy street corners, attracting hungry passersby with mouthwatering aromas. One of the best things about street food is its accessibility. It’s quick, affordable, and often made fresh right in front of you. Whether it’s a steaming bowl of pho in Vietnam or a crispy samosa in India, there’s something for everyone to enjoy. Street food also brings people together.'),
 Document(metadata={}, page_content='Friends and families gather around food stalls, sharing dishes and stories. It’s a social experience that transcends language and culture, creating connections over a shared love of good eats. Finally, street food is constantly evolving. Chefs are experimenting with traditional recipes, adding modern twists and fusion flavors.'),
 Document(m